In [3]:
import os
import json
import datetime
from dateutil import parser
import glob
import json
from dataclasses import dataclass, field
from dacite import from_dict, Config
import pandas as pd

from googletrans import Translator
CREDENTIALS_FILE = "../creds.txt"
USERDIR = "../datasets/users"

USERS_FILENAME = "users.pkl"
TWEETS_FILENAME = "tweets.pkl"
CONTROL_USERS_FILENAME = "control_" + USERS_FILENAME

CONTROL_TWEETS_FILENAME = "control_" + TWEETS_FILENAME

DIR = "../datasets/pandas/"

EXPS = ["hk",  "control", "nyc"]

# USERS_FILEPATH = os.path.join(DIR, USERS_FILENAME)
# CONTROL_USERS_FILEPATH = os.path.join(DIR, CONTROL_USERS_FILENAME)

# TWEETS_FILEPATH = os.path.join(DIR, TWEETS_FILENAME)
# CONTROL_TWEETS_FILEPATH = os.path.join(DIR, CONTROL_TWEETS_FILENAME)


In [6]:
data = {}
for exp in EXPS:
    data[exp] = {
        "users": pd.read_pickle(os.path.join(DIR, exp, USERS_FILENAME)),
        "tweets": pd.read_pickle(os.path.join(DIR, exp, TWEETS_FILENAME))
    }

print(data)
# users_df = pd.read_pickle(USERS_FILEPATH)
# control_users_df = pd.read_pickle(CONTROL_USERS_FILEPATH)

# tweets_df = pd.read_pickle(TWEETS_FILEPATH)
# control_tweets_df = pd.read_pickle(CONTROL_TWEETS_FILEPATH)



{'hk': {'users':                         id      screen_name                    name  \
0               2998716344    YumaOhnishiSM  大西結真☆熟女系セクシー女優と性感セラピスト   
1                216958914  andymukherjee70          Andy Mukherjee   
2       926387481532022784       crackhcead                   Lucas   
3                 40284750       chinmoylad             Chinmoy Lad   
4       721053264259715072        omydeoro_                Omeeeh 🎃   
...                    ...              ...                     ...   
35853  1129767015982481408  NoellaDecarter1             Hempel Zang   
35854           2737435050    MochiCatZakka         Mochi Cat Zakka   
35855  1029016457592500225    Explorerdora2           Explorer_dora   
35856           4155562332       Jo_Lumiere          Joanne Lumière   
35857   788977167069646848           occur0                 monarch   

                                             description  \
0      海外で活動のセクシー女優 @JWinstonTS 日本芸名。日本ではR18エンターテイメント...   
1      Blo

In [23]:
def lang_stats(df, lang):
    total = sum(list(df["lang"] == lang))
    print(f"{total} of {lang}")
    
def summary_stats(datum):
    users = datum["users"]
    users = users[~users["location"].isnull()]
    user_ids = set(users["id"].unique())
    tweets = datum["tweets"]
    tweets = tweets[tweets["user_id"].isin(user_ids)]
    print(f"{len(users)} users")
    print(f"{len(tweets)} tweets")
    for lang in ["en", "zh", "ja"]:
        lang_stats(tweets, lang)
    
for exp, datum in data.items():
    print(f"EXPERIMENT: {exp}")
    summary_stats(datum)

EXPERIMENT: hk
35858 users
264988 tweets
111530 of en
23893 of zh
48121 of ja
EXPERIMENT: control
18746 users
18797 tweets
6332 of en
32 of zh
3266 of ja
EXPERIMENT: nyc
18035 users
132859 tweets
115241 of en
95 of zh
1028 of ja


In [15]:
def deleted_stats(df):
    total = len(df)
    df = df[~df["location"].isnull()]
    n_protected = sum(df["protected"] == True)
    n_deleted = sum(df["deleted"] == True)
    
    print(f"{(n_protected)/total} Protected")
    print(f"{(n_deleted)/total} Deleted")
    print(f"{(n_protected+n_deleted)/total} Protected or Deleted")

print("% Accounts deleted or protected since")
for exp, datum in data.items():
    print(f"EXPERIMENT: {exp}")
    deleted_stats(datum["users"])

% Accounts deleted or protected since
EXPERIMENT: hk
0.08254782754197111 Protected
0.13706843661107704 Deleted
0.21961626415304814 Protected or Deleted
EXPERIMENT: control
0.058490501688209816 Protected
0.13502267999045053 Deleted
0.19351318167866036 Protected or Deleted
EXPERIMENT: nyc
0.04602162461879678 Protected
0.09198780149708899 Deleted
0.13800942611588576 Protected or Deleted


In [16]:
def deleted_tweets_stats(df):
    total = len(df)
    n_deleted = sum(df["deleted"] == True)
    n_user_deleted = sum(df["user_deleted"] == True)
    n_user_protected = sum(df["user_protected"] == True)

    print(f"{(n_deleted)/total} deleted, user public")
    print(f"{(n_deleted+n_user_deleted+n_user_protected)/total} inaccessible")

def deleted_tweets_stats_lang(df, lang):
    print(f"Lang: {lang}")
    deleted_tweets_stats(df[df["lang"] == lang])

print("% Accounts deleted or protected since")
for exp, datum in data.items():
    print(f"EXPERIMENT: {exp}")
    deleted_tweets_stats(datum["tweets"])
    deleted_tweets_stats_lang(datum["tweets"], "en")
    deleted_tweets_stats_lang(datum["tweets"], "zh")
    deleted_tweets_stats_lang(datum["tweets"], "ja")


% Accounts deleted or protected since
EXPERIMENT: hk
0.34157395806602564 deleted, user public
0.5576177034431747 inaccessible
Lang: en
0.3406168743835739 deleted, user public
0.5591500044830987 inaccessible
Lang: zh
0.4252291466119784 deleted, user public
0.7490478382789938 inaccessible
Lang: ja
0.27254213337212446 deleted, user public
0.43544398495459363 inaccessible
EXPERIMENT: control
0.15060015641470298 deleted, user public
0.4566969295113741 inaccessible
Lang: en
0.16929133858267717 deleted, user public
0.4508967629046369 inaccessible
Lang: zh
0.23404255319148937 deleted, user public
0.574468085106383 inaccessible
Lang: ja
0.10722222222222222 deleted, user public
0.38462962962962965 inaccessible
EXPERIMENT: nyc
0.28618309636531963 deleted, user public
0.4513130461617203 inaccessible
Lang: en
0.28340607943353496 deleted, user public
0.44502390642219347 inaccessible
Lang: zh
0.3473684210526316 deleted, user public
0.631578947368421 inaccessible
Lang: ja
0.28599221789883267 deleted, 

In [17]:
china_tweets = tweets_df[tweets_df["text"].str.contains("中国")]
china_tweets = china_tweets[~china_tweets["user_protected"]]
china_tweets = china_tweets[~china_tweets["user_deleted"]]

print(sum(china_tweets["deleted"])/len(china_tweets))


NameError: name 'tweets_df' is not defined

In [18]:

import nltk
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
import joblib
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
import string

In [19]:
import re
eng_tweets = tweets_df[tweets_df["lang"] == "en"]

eng_tweets = eng_tweets[~eng_tweets["user_protected"]]
eng_tweets = eng_tweets[~eng_tweets["user_deleted"]]
eng_tweets["label"] = eng_tweets.apply(lambda r: int(r.deleted), axis=1)
eng_tweets = eng_tweets[["text", "label"]]
not_deleted = eng_tweets['label'][eng_tweets.label == 0]
deleted = eng_tweets['label'][eng_tweets.label == 1]
print(len(not_deleted)/len(eng_tweets))
print(len(deleted))

# helper function to clean tweets
def processTweet(tweet):
    # Remove HTML special entities (e.g. &amp;)
    tweet = re.sub(r'\&\w*;', '', tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','',tweet)
    # Remove tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    # To lowercase
    tweet = tweet.lower()
    # Remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    # Remove hashtags
    tweet = re.sub(r'#\w*', '', tweet)
    # Remove Punctuation and split 's, 't, 've with a space for filter
    tweet = re.sub(r'[' + string.punctuation.replace('@', '') + ']+', ' ', tweet)
    # Remove words with 2 or fewer letters
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet)
    # Remove whitespace (including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    # Remove single space remaining at the front of the tweet.
    tweet = tweet.lstrip(' ') 
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    tweet = ''.join(c for c in tweet if c <= '\uFFFF') 
    return tweet

eng_tweets["text"] = eng_tweets["text"].apply(processTweet)
display(eng_tweets)

NameError: name 'tweets_df' is not defined

In [160]:
data = eng_tweets.copy()

# Run Train Data Through Pipeline analyzer=text_process
# uncomment below to train on a larger dataset but it's very slow for a slower machine.# X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(data['text'][:5000], data['label'][:5000], test_size=0.2)# create pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(strip_accents='ascii',
                            stop_words='english',
                            lowercase=True)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', LogisticRegression()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])# this is where we define the values for GridSearchCV to iterate over
parameters = {'bow__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              #'classifier__alpha': (1e-2, 1e-3),
             }# do 10-fold cross validation for each of the 6 possible combinations of the above params
grid = GridSearchCV(pipeline, cv=10, param_grid=parameters, verbose=1)
grid.fit(X_train,y_train)# summarize results
print("\nBest Model: %f using %s" % (grid.best_score_, grid.best_params_))
print('\n')
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean: %f Stdev:(%f) with: %r" % (mean, stdev, param))

Fitting 10 folds for each of 4 candidates, totalling 40 fits

Best Model: 0.886250 using {'bow__ngram_range': (1, 1), 'tfidf__use_idf': True}


Mean: 0.886250 Stdev:(0.001250) with: {'bow__ngram_range': (1, 1), 'tfidf__use_idf': True}
Mean: 0.886250 Stdev:(0.001250) with: {'bow__ngram_range': (1, 1), 'tfidf__use_idf': False}
Mean: 0.886250 Stdev:(0.001250) with: {'bow__ngram_range': (1, 2), 'tfidf__use_idf': True}
Mean: 0.886250 Stdev:(0.001250) with: {'bow__ngram_range': (1, 2), 'tfidf__use_idf': False}


In [190]:
import numpy as np
lr = grid.best_estimator_.named_steps["classifier"]
count_vect = grid.best_estimator_.named_steps["bow"]

results = zip(count_vect.get_feature_names(), lr.coef_[0])
#print(sorted(list(results), key= lambda x: x[1]))
print([x[0] for x in sorted(list(results), key= lambda x: x[1], reverse=False)][:100])
results = zip(count_vect.get_feature_names(), lr.coef_[0])

print([x[0] for x in sorted(list(results), key= lambda x: x[1], reverse=True)][:100])



['insurance', 'china', 'tls', 'flight', 'departed', 'insurers', 'life', 'big', 'asia', 'kong', 'thanks', 'watched', 'cathay', 'working', 'wanna', 'australia', 'today', 'teaser', 'miss', 'malaysia', 'cute', 'pacific', 'week', 'song', 'wrong', 'pop', 'game', 'episode', 'tell', 'nct', 'reinsurance', 'trump', 'lol', 'indonesia', 'job', 'trying', 'want', 'public', 'learn', 'risk', 'better', 'govt', 'read', 'realize', 'matter', 'hands', 'local', 'stand', 'writes', 'feeling', 'color', '000', 'problem', 'doing', 'enjoy', 'beautiful', 'hong', 'costs', 'school', 'dreams', '1st', 'fan', 'oht', 'catch', 'gift', 'fine', 'changed', 'hobi', 'used', 'mood', 'second', 'philippines', 'waste', 'health', 'focus', 'psalm', 'version', 'begin', 'adn', 'save', 'drop', 'airlines', 'taiwan', 'sector', 'pray', 'capital', 'problems', 'attention', 'concert', 'inside', 'singing', 'india', 'sweetest', 'countdown', 'ohmygod', 'hear', 'hurt', 'does', '2014', 'insurer']
['yes', 'don', 'welcome', 'jungkook', 'music', 'g

In [159]:
import numpy as np
nb = grid.best_estimator_.named_steps["classifier"]
neg_class_prob_sorted = nb.feature_log_prob_[0, :].argsort()[::-1]
pos_class_prob_sorted = nb.feature_log_prob_[1, :].argsort()[::-1]

count_vect = grid.best_estimator_.named_steps["bow"]
print(np.take(count_vect.get_feature_names(), neg_class_prob_sorted[:100]))
print(np.take(count_vect.get_feature_names(), pos_class_prob_sorted[:100]))


['love' 'thank' 'life' 'don' 'insurance' 'just' 'flight' 'departed'
 'china' 'like' 'people' 'new' 'tls' 'time' 'insurers' 'good' 'know'
 'happy' 'think' 'heart' 'want' 'day' 'way' 'today' '2019' 'make' 'cathay'
 'best' 'hong' 'kong' 'asia' 'let' 'pacific' 'look' 'world' 'global' 'say'
 'adn' 'better' 'live' 'year' 'need' 'things' 'big' 'https' 'really'
 'hope' 'aldubnation' 'right' 'smile' 'said' 'stop' 'going' 'eyes'
 'change' 'thanks' 'did' 'trump' 'years' 'japan' 'feel' 'singapore'
 'hello' 'album' 'work' 'person' 'bts' 'doesn' 'airlines' 'ready'
 'indonesia' 'video' 'australia' 'reinsurance' 'come' 'doing' 'episode'
 'deserve' 'support' 'true' 'risk' 'man' 'malaysia' 'watched' 'india'
 'god' 'happiness' 'working' 'hard' 'miss' 'looks' 'long' 'does' 'local'
 'growth' 'face' 'real' 'philippines' 'mind' 'thing']
['love' 'like' 'don' 'time' 'just' 'people' 'let' 'happy' 'new' 'thank'
 'year' 'good' 'need' 'know' 'way' 'think' 'music' 'bts' 'day' 'feel'
 'video' 'yes' 'giveaway' 'great

In [105]:

print("% Accounts deleted or protected since")
print("EXPERIMENT:")
deleted_tweets_stats(tweets_df)
print("CONTROL:")
deleted_tweets_stats(control_tweets_df)

print("english")
deleted_tweets_stats(tweets_df[tweets_df["lang"] == "en"])
print("english-control")
deleted_tweets_stats(control_tweets_df[control_tweets_df["lang"] == "en"])

print("zh")
deleted_tweets_stats(tweets_df[tweets_df["lang"] == "zh"])
print("zh-control")
deleted_tweets_stats(control_tweets_df[control_tweets_df["lang"] == "zh"])

print("ja")
deleted_tweets_stats(tweets_df[tweets_df["lang"] == "ja"])
print("ja-control")
deleted_tweets_stats(control_tweets_df[control_tweets_df["lang"] == "ja"])

% Accounts deleted or protected since
EXPERIMENT:
0.34157395806602564 deleted, user public
0.5576177034431747 inaccessible
CONTROL:
0.28618309636531963 deleted, user public
0.4513130461617203 inaccessible
english
0.3406168743835739 deleted, user public
0.5591500044830987 inaccessible
english-control
0.28340607943353496 deleted, user public
0.44502390642219347 inaccessible
zh
0.4252291466119784 deleted, user public
0.7490478382789938 inaccessible
zh-control
0.3473684210526316 deleted, user public
0.631578947368421 inaccessible
ja
0.27254213337212446 deleted, user public
0.43544398495459363 inaccessible
ja-control
0.28599221789883267 deleted, user public
0.4377431906614786 inaccessible


In [174]:
print(TWEET_QUERY_RESULTS)

tweets = {}
rts = 0
with open(TWEET_QUERY_RESULTS, "r") as f:
    for line in f:
        line = line.strip()
        data = json.loads(line)
        if data["found"] and data["text"].startswith("RT @"):
            rts += 1
            continue
        tweets[data["id"]] = data
        
rts_control = 0
tweets_control = {}
with open(CONTROL_TWEET_QUERY_RESULTS, "r") as f:
    for line in f:
        line = line.strip()
        data = json.loads(line)
        if data["found"] and data["text"].startswith("RT @"):
            rts_control += 1
            continue
        tweets_control[data["id"]] = data

# TODO: CONTROL FOR DELETED RTS
print("% Tweets of public users deleted since")
print("EXPERIMENT:")
print(1-sum([result["found"] for result in users.values()])/float(len(users)))
print("CONTROL:")
print(1-sum([result["found"] for result in users_control.values()])/float(len(users_control)))

print("==============")
print("% RTs that still exist today")
print("EXPERIMENT:")
print(rts / float(rts + len(tweets)))
print("CONTROL:")
print(rts_control / float(rts_control + len(tweets)))

NameError: name 'TWEET_QUERY_RESULTS' is not defined

,id,text,created_at,lang,source,retweeted,user_id,from_ia,user_deleted,deleted,user_protected,last_query_date
21,1097261639550693376,"Now think of the going rates for mfg units, po...",2019-02-17 22:28:55+00:00,en,"<a href=""http://twitter.com/download/iphone"" r...",False,216958914,True,False,False,False,2022-01-02 14:40:28.657956
22,1101457126336032768,"@viv_chand The “Cam chho? Majama” letter, you ...",2019-03-01 12:20:17+00:00,en,"<a href=""http://twitter.com/download/iphone"" r...",False,216958914,True,False,False,False,2021-12-27 11:48:16.470983
23,1101412440242192384,Call Clean-Chit Cam!,2019-03-01 09:22:43+00:00,en,"<a href=""http://twitter.com/download/iphone"" r...",False,216958914,True,False,False,False,2021-12-27 11:48:46.431349
24,1161954290866372608,Terrific piece by @menakadoshi. Do read.,2019-08-15 10:54:25+00:00,en,"<a href=""http://twitter.com/download/iphone"" r...",False,216958914,True,False,False,False,2021-12-27 11:48:32.794388
25,1117287268509552640,RT @WongDouglas: Assange is that new 21st cent...,2019-04-14 04:43:37+00:00,en,"<a href=""http://twitter.com/download/iphone"" r...",False,216958914,True,False,False,False,2021-12-27 11:47:17.066868
...,...,...,...,...,...,...,...,...,...,...,...,...
264982,1146457272916201473,Check out @CryptoMiningBiz's audit! https://t....,2019-07-03 16:34:48+00:00,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,815559798141747201,True,False,False,False,2022-01-02 14:59:02.465606
264984,1146450067093479424,RT @antd: The “helmets” hand signal ripples on...,2019-07-03 16:06:10+00:00,en,"<a href=""http://twitter.com/download/iphone"" r...",False,2737435050,True,False,False,False,2022-01-02 15:01:02.994889
264985,1146456320804966401,RT @WorldOfDakota: NEW: Dakota on the set of “...,2019-07-03 16:31:01+00:00,en,"<a href=""http://twitter.com/download/iphone"" r...",False,1029016457592500225,True,False,False,True,None
264986,1146463199438315521,RT @OsasWrite: This is Twitter users when Face...,2019-07-03 16:58:21+00:00,en,"<a href=""http://twitter.com/download/iphone"" r...",False,4155562332,True,False,True,False,2022-01-02 14:34:28.691055


In [32]:
control_tweets_df["lang"].value_counts()

en     115241
und      9017
es       2453
ja       1028
pt        742
tl        487
fr        484
ar        482
in        372
ko        317
it        209
pl        204
ht        198
de        189
tr        151
nl        131
et        125
ca        100
zh         95
cs         94
th         82
hi         82
ru         80
da         70
cy         54
sv         53
sr         50
ro         49
no         47
eu         24
hu         22
lt         15
bg         15
fi         14
is         14
lv         12
sl         11
ur         11
fa         11
vi         10
iw          7
uk          2
el          2
my          1
bn          1
mr          1
Name: lang, dtype: int64